---
## 词嵌入

词嵌入在PyTorch中通过函数 `nn.Embdding(m, n)` 来实现的。`m`表示所有的单词数目，n表示词嵌入的维度

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim

In [2]:
word_to_ix = {'hello' : 0, 'world' : 1}
embeds = nn.Embedding(2, 5)
hello_idx = torch.LongTensor([word_to_ix['hello']])
hello_idx = Variable(hello_idx)
hello_embed = embeds(hello_idx)
print(hello_embed)

Variable containing:
 0.8044  0.0947 -0.2797  0.4324 -0.5038
[torch.FloatTensor of size 1x5]



---
## N Gram 模型

In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [4]:
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2])
           for i in range(len(test_sentence) - 2)]

In [5]:
vocb = set(test_sentence)
word_to_ix = {word : i for i, word in enumerate(vocb)}
idx_to_word = {word_to_ix[word] : word for word in word_to_ix}

In [6]:
class NgramModel(nn.Module) :
    def __init__(self, vocb_size, context_size, n_dim) :
        super().__init__()
        self.n_word = vocb_size
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size * n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)
    
    def forward(self, x) :
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        log_prob = F.log_softmax(out, 1)
        return log_prob

In [7]:
net = NgramModel(len(vocb), CONTEXT_SIZE, EMBEDDING_DIM)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-2, weight_decay=1e-5)

In [9]:
epoches = 200
for epoch in range(epoches) :
    train_loss = 0
    for word, label in trigram : 
        word = Variable(torch.LongTensor([word_to_ix[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_ix[label]]))
        out = net(word)
        loss = criterion(out, label)
        train_loss += loss.data[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 20 == 0 :
        print('epoch: {}, Loss : {:.6f}'.format(epoch + 1, train_loss / len(trigram)))

epoch: 20, Loss : 0.809687
epoch: 40, Loss : 0.152495
epoch: 60, Loss : 0.095547
epoch: 80, Loss : 0.076622
epoch: 100, Loss : 0.066594
epoch: 120, Loss : 0.060094
epoch: 140, Loss : 0.055386
epoch: 160, Loss : 0.051820
epoch: 180, Loss : 0.049030
epoch: 200, Loss : 0.046760


In [10]:
net = net.eval()

In [11]:
word, label = trigram[19]
print('input: {}'.format(word))
print('input: {}'.format(label), end ="\n\n")

word = Variable(torch.LongTensor([word_to_ix[i] for i in word]))
out = net(word)

pred_label_idx = out.max(1)[1].data[0]
print(pred_label_idx)

predict_word = idx_to_word[pred_label_idx]
print('real word is "{}", predicted word is "{}"'.format(label, predict_word))

input: ('so', 'gazed')
input: on

67
real word is "on", predicted word is "on"
